# Hashing
Message -> Hash Function -> Hash Value(number)

variable-length input -> fixed-length output

## Hash function properties
- deterministic behaviour - for given input produces the same output
- fixed-length hash values 
- avalanche effect - when small difference in message results in large differences between hash values

eg. `hash()` function used to hash keys in Python dicts. For cryptographic hash function additional properties must be met.


## Cryptographic hash function properties 
- One-way function property - must be difficult to identify input from given output
- Weak collision resistance - given one message it's infeasible to identify second message that computes to the same hash value
- Strong collision resistance - it's infeasible to find any collision at all


In [ ]:
print(hash("dupa"))
print(hash("dupa"))
print(hash("dupa2"))

## Cryptographic hashing in Python

In [ ]:
import hashlib

# list of all hash algorithms
print(hashlib.algorithms_available)

# list of hash algorithms available on all platforms
print(hashlib.algorithms_guaranteed)

`MD5` and `SHA1` are no longer suitable for data integrity.

Use `SHA2` (standard) or `SHA3` (new standard) or `Blake` (fast). Most common is `SHA256`.


In [ ]:
import hashlib

# Python 3 strings saved in unicode code points (UTF-8)
# Hash function argument must be bytes; strings must be encoded to become bytes
hash1 = hashlib.sha256(b'duuupa')
hash2 = hashlib.sha256('duuupa'.encode())
print(hash1.digest_size, 'bytes')

# Hash value in str
print(hash1.hexdigest())
print(hash2.hexdigest())

# Hash value in bytes
print(hash1.digest())
print(hash2.digest())


In [ ]:
from hashlib import sha256

# Chunked hash generation using update()
many = sha256()
many.update(b'm')
many.update(b'e')
many.update(b's')
many.update(b's')
print(many.hexdigest())

print(sha256(b'mess').hexdigest())

## Checksum functions
Checksums (eg. CRC, Adler-32) are fast and have insufficient collision resistance - can be used for error detection.

Hash functions (SHA2 family, SHA3 family) are slower and have sufficient collision resistance - can be used for testing data integrity.

In [ ]:
import zlib

# CRC checksum collision
print(zlib.crc32(b'gnu'))
print(zlib.crc32(b'codding'))

# Adler-32 checksum no collision
print(zlib.adler32(b'gnu'))
print(zlib.adler32(b'codding'))


# Keyed hashing
## Data authentication
Data authentication (who authored the change?) - requires __key__ and a __keyed hash function__

### Key generation
Key can be in form of:
- random number - sequence of random numbers
- passphrase - sequence of random words

### Random number
Keys that are hard to remember

Use `secrets` module. Do not use `random` module.

In [ ]:
import os

# random secret generation - 16 bytes
print(os.urandom(16))

In [ ]:

from secrets import token_bytes, token_hex, token_urlsafe

# random secret generation - 16 bytes
print(token_bytes(16))
print(token_hex(16))
print(token_urlsafe(16))

### Passphrases
Keys that are easy to remember

In [ ]:
import secrets
from pathlib import Path

words = Path('wordlist.txt').read_text().splitlines()
passphrase = ' '.join(secrets.choice(words) for i in range(4))
print(passphrase)

### Keyed hashing
__Keyed hash functions__ use _key and message_ to produce _hash value_. The same message with different key is hashed to different hash value. Only some functions can do that by default like `blake2b`.

In [ ]:
from hashlib import blake2b

m = b'message'
x = b'key x'
y = b'key y'

print(blake2b(m, key=x).hexdigest())
print(blake2b(m, key=y).hexdigest())

## HMAC functions
Hash-based Message Authentication Code.

HMAC functions allow any generic hashing function to become keyed hash functions. It has 3 inputs:
- message
- key
- hash function

In [ ]:
import hashlib
import hmac

xx = hmac.new(key=b'key', msg=b'message', digestmod=hashlib.sha3_256)
print(xx.name)  # protocol name prefixed with HMAC
print(xx.hexdigest())

### Data authentication between parties

In [ ]:
import hashlib
import hmac
import json
hmac_sha256 = hmac.new(b'shared_key', digestmod=hashlib.sha256)
message = b'from Bob to Alice'
hmac_sha256.update(message)
hash_value = hmac_sha256.hexdigest()
authenticated_msg = {
    'message': list(message),
    'hash_value': hash_value, 
    }
outbound_msg_to_alice = json.dumps(authenticated_msg)

In [ ]:
import hashlib
import hmac
import json
authenticated_msg = json.loads(inbound_msg_from_bob)
message = bytes(authenticated_msg['message'])
hmac_sha256 = hmac.new(b'shared_key', digestmod=hashlib.sha256)
hmac_sha256.update(message)
hash_value = hmac_sha256.hexdigest()
if hash_value == authenticated_msg['hash_value']:
    print('trust message')

## Timing attacks
String comparison of 2 hash values is faster if they are different (evaluates to False faster). Attacker can measure response time to invalid hash (_side channel attack_).

To mitigate this problem use length-constant time or random time. Always use `compare_digest()` for hashes.


In [ ]:
from hmac import compare_digest

compare_digest('abc', 'abcd')

# Symmetric encryption
Encryption - process of obfuscating plaintext into ciphertext using cipher (encryption algorithm) together with key. Encryption ensures confidentiality.

Decryption - reverse process

Fernet guarantees that a message encrypted using it cannot be manipulated or read without the key. Fernet is an implementation of symmetric (also known as “secret key”) authenticated cryptography. Fernet also has support for implementing key rotation via MultiFernet.

In [ ]:
from cryptography.fernet import Fernet
key = Fernet.generate_key()  # random 32 byte key in bytes format
print(key)
fernet = Fernet(key)  # general purpose class initialized with key

# token - combined ciphertext HMAC hash value from that ciphertext (confidentiality + message authenticity???)
token = fernet.encrypt(b'duuupa')
token

## Key rotation
Key rotation - retire one key with another. It means decrypting all ciphertext with old key and reencrypting them with new key

In [ ]:
from cryptography.fernet import Fernet, MultiFernet

# Encrypting with old key
old_key = Fernet.generate_key()
old_fernet = Fernet(old_key)
old_token1 = old_fernet.encrypt(b'dupa')
old_token2 = old_fernet.encrypt(b'krowa')

# Creating new key
new_key = Fernet.generate_key()
new_fernet = Fernet(new_key)

multi_fernet = MultiFernet([new_fernet, old_fernet])
# List of tokens encrypted with old key
old_tokens = [old_token1, old_token2]
print(old_tokens)
# Decrypting old tokens and reencrypting them with new key
new_tokens = [multi_fernet.rotate(t) for t in old_tokens]
print(new_tokens)

#replace_old_tokens(new_tokens)
#replace_old_key_with_new_key(new_key)
#del old_key

# Decrypt after rotation
for new_token in new_tokens:
    plaintext = new_fernet.decrypt(new_token)
    print(plaintext)

### Categories of encryption algorithms
- block ciphers (AES - 128, 192, 256) - encrypts fixed-length blocks
- stream ciphers (ChaCha) - processes stream of bytes

### Symmetric encryption modes:
- ECB (Electronic CodeBook) - for less than block-length plaintext fills in with padding
- CBC (Cipher Block Chaining) - with initialization vector (IV)
- GCM (Galois Counter Mode)
- others

In [ ]:
# ECB example
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


key = b'key must be 128, 196 or 256 bits'
cipher = Cipher(
    algorithms.AES(key),  # AES cipher
    modes.ECB(),  # ECB mode
    backend=default_backend()  # OpenSSL
    )

encryptor = cipher.encryptor()
plaintext = b'block size = 128'  # signle block of plaintext

ciphertext = encryptor.update(plaintext) + encryptor.finalize()  # single block of ciphertext
print(ciphertext)

### Summary
- ecryption ensures confidentiality
- symmetric encryption algorithms use the same key for encryption and decryption

# Asymmetric encryption

## RSA public-key encryption
 
RSA - public-key cryptosystem that involves 4 steps:
1. key generation
1. key distribution
1. encryption
1. decryption


Create private key:
```
openssl genpkey -algorithm RSA -out private_key.pem -pkeyopt rsa_keygen_bits:2048
```

Create public key:

```
openssl rsa -pubout -in private_key.pem -out public_key.pem
```

 
Public-key cryptography allows:
- encrypting with public key and only private key can decrypt - ensures data confidentiality
- encrypting with private key and only public key can decrypt - ensures data authenticity
 
 
## Nonrepudiation
When system prevents a participant from denying their actions. Assymentric encryption alone
 
Assymentric encryption alone does not ensure nonrepudiation because eg.:
- anyone with public key can create message and claim it's from someone else
- anyone who received message encrypted with private key can say he is the author of the message
 
In order to achieve nonrepudiation a digital signature is needed.
 
## RSA Digital signatures
Digital signature allows __anyone__ to check:
- who sent the message?
- has the message been modified in transit?
 
Digital signature:
- is unique to signer
- can be used to legally bind the signer to a contract
- is difficult to forge
 
Digital signature combines hashing with public-key encryption.
 
Sender sends message hash encrypted with private key along with message itself
Receiver can decrypt the hash with public key and compare it with computed hash of a message
 
RSA digital signing uses different padding scheme than RSA encryption.
 
## Elliptic-curve digital signatures
Cryptosystem similar to RSA with following characteristics:
- uses key-pair for signing data and verifying signatures
- private key cannot decrypt what public key encrypted
- faster signing data and verifying signatures than RSA
- less lines of code to use
- ECC = Elliptic Curve Cryptography; ECDSA = Elliptic Curve Digital Signature Algorithm
 
 
## Summary
- Hashing ensures data integrity and data authentication.
- Encryption ensures confidentiality.
- Digital signatures ensure nonrepudiation

# TLS - Transport Layer Security
- ensures data integrity, data authentication, confidentiality, and nonrepudiation
- point-to-point client/server protocol
 
## TLS handshake
Typically client initiates handshake(s) with server. Handshake objective is to perform:
- cipher suite negotiation
- key exchange
- server authentication
 
### Cipher suite negotiation
Client and server must first agree on common set of algorithms known as cipher suite. 
Cipher suite defines encryption and hashing algorithms.
Each TLS version defines different cipher suites. 


TLS 1.2 defines 37 cipher suites


TLS 1.3 defines 5 cipher suites:
- TLS_AES_128_CCM_8_SHA256
- TLS_AES_128_CCM_SHA256
- TLS_AES_128_GCM_SHA256
- TLS_AES_256_GCM_SHA384
- TLS_CHACHA20_POLY1305_SHA256


TLS_AES_128_GCM_SHA256:
- TLS_- common prefix
- AES_128_GCM - symmetric encryption with 128bit key in GCM mode
- _SHA256 - hashing


TLS ensures confidentiality with symmetric encryption (it's more efficient than assymetric encryption).
 
### Key exchange
Client and server must exchange a key that will be used with cipher suite for symmetric encryption.

In TLS 1.2 key-distribution problem is solved by Diffie-Hellman key exchange or STATIC RSA key-exchange method (worse)

In TLS 1.3 key-distribution problem is solved by Diffie-Hellman key exchange (Perfect Forward Secrecy)

Diffie-Hellman key-exchange is one-roundtrip key-exchange algorithm that results in both nodes independently computing shared secret key for symmetric encryption.
 
 
### Server authentication
Server authenticates itself to client (browser) using public-key certificate that proves public-key ownership. Public-key certificate is created and issued by Certificate Authority. Applciation for public-key certificate is called CSR - Certificate Signing Request.
 


In [ ]:
# Certificate in format X.509
import ssl
address = ('wikipedia.org', 443)
certificate = ssl.get_server_certificate(address)
print(certificate)

To decode above certificate use 

```
$ openssl.exe x509 -in server.crt -text -noout
Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            02:7d:94:1b:29:2c:db:2e:da:f9:93:11:18:53:74:3e
        Signature Algorithm: ecdsa-with-SHA384
        Issuer: C = US, O = DigiCert Inc, CN = DigiCert TLS Hybrid ECC SHA384 2020 CA1
        Validity
            Not Before: Oct 19 00:00:00 2021 GMT
            Not After : Nov 17 23:59:59 2022 GMT
        Subject: C = US, ST = California, L = San Francisco, O = "Wikimedia Foundation, Inc.", CN = *.wikipedia.org
        Subject Public Key Info:
            Public Key Algorithm: id-ecPublicKey
                Public-Key: (256 bit)
                pub:
                    04:e8:50:2c:d0:d2:4e:a2:b1:92:aa:b6:73:0f:cf:
                    a0:b4:57:e5:c2:c0:7c:ae:6e:55:91:4a:a6:94:67:
                    fa:a5:f8:b0:3f:46:ac:23:52:b4:48:3b:64:64:fb:
                    ea:cd:e9:e4:fb:8f:10:a7:f4:e8:23:ba:95:29:6e:
                    ef:ca:72:bb:83
                ASN1 OID: prime256v1
                NIST CURVE: P-256
        X509v3 extensions:
            X509v3 Authority Key Identifier:
                keyid:0A:BC:08:29:17:8C:A5:39:6D:7A:0E:CE:33:C7:2E:B3:ED:FB:C3:7A

            X509v3 Subject Key Identifier:
                89:25:8E:F1:85:F9:7F:5C:D1:87:14:98:4A:5E:F4:1C:32:1D:51:44
            X509v3 Subject Alternative Name:
                DNS:*.wikipedia.org, DNS:wikimedia.org, DNS:mediawiki.org, DNS:wikibooks.org, DNS:wikidata.org, DNS:wikinews.org, DNS:wikiquote.org, DNS:wikisource.org, DNS:wikiversity.org, DNS:wikivoyage.org, DNS:wiktionary.org, DNS:wikimediafoundation.org, DNS:w.wiki, DNS:wmfusercontent.org, DNS:*.m.wikipedia.org, DNS:*.wikimedia.org, DNS:*.m.wikimedia.org, DNS:*.planet.wikimedia.org, DNS:*.mediawiki.org, DNS:*.m.mediawiki.org, DNS:*.wikibooks.org, DNS:*.m.wikibooks.org, DNS:*.wikidata.org, DNS:*.m.wikidata.org, DNS:*.wikinews.org, DNS:*.m.wikinews.org, DNS:*.wikiquote.org, DNS:*.m.wikiquote.org, DNS:*.wikisource.org, DNS:*.m.wikisource.org, DNS:*.wikiversity.org, DNS:*.m.wikiversity.org, DNS:*.wikivoyage.org, DNS:*.m.wikivoyage.org, DNS:*.wiktionary.org, DNS:*.m.wiktionary.org, DNS:*.wikimediafoundation.org, DNS:*.wmfusercontent.org, DNS:wikipedia.org
            X509v3 Key Usage: critical
                Digital Signature
            X509v3 Extended Key Usage:
                TLS Web Server Authentication, TLS Web Client Authentication
            X509v3 CRL Distribution Points:

                Full Name:
                  URI:http://crl3.digicert.com/DigiCertTLSHybridECCSHA3842020CA1-1.crl

                Full Name:
                  URI:http://crl4.digicert.com/DigiCertTLSHybridECCSHA3842020CA1-1.crl

            X509v3 Certificate Policies:
                Policy: 2.23.140.1.2.2
                  CPS: http://www.digicert.com/CPS

            Authority Information Access:
                OCSP - URI:http://ocsp.digicert.com
                CA Issuers - URI:http://cacerts.digicert.com/DigiCertTLSHybridECCSHA3842020CA1-1.crt

            X509v3 Basic Constraints: critical
                CA:FALSE
            CT Precertificate SCTs:
                Signed Certificate Timestamp:
                    Version   : v1 (0x0)
                    Log ID    : 29:79:BE:F0:9E:39:39:21:F0:56:73:9F:63:A5:77:E5:
                                BE:57:7D:9C:60:0A:F8:F9:4D:5D:26:5C:25:5D:C7:84
                    Timestamp : Oct 19 16:24:26.110 2021 GMT
                    Extensions: none
                    Signature : ecdsa-with-SHA256
                                30:45:02:20:13:2B:9F:D4:BF:5D:66:83:C7:35:3A:0E:
                                8B:73:E7:EA:EA:41:D2:0F:7F:E0:25:DB:80:22:84:1C:
                                A4:E7:A1:FE:02:21:00:83:55:AD:D4:1D:03:23:2C:4E:
                                1A:6E:E6:FD:73:8F:1A:AF:90:4E:FE:1E:50:04:64:C4:
                                4F:DA:42:B9:21:C1:02
                Signed Certificate Timestamp:
                    Version   : v1 (0x0)
                    Log ID    : 51:A3:B0:F5:FD:01:79:9C:56:6D:B8:37:78:8F:0C:A4:
                                7A:CC:1B:27:CB:F7:9E:88:42:9A:0D:FE:D4:8B:05:E5
                    Timestamp : Oct 19 16:24:26.102 2021 GMT
                    Extensions: none
                    Signature : ecdsa-with-SHA256
                                30:45:02:21:00:EC:ED:B1:2D:A6:3E:BC:5D:AE:0F:48:
                                D5:70:5B:A4:E9:63:32:26:13:86:C6:D4:16:66:68:9C:
                                64:67:60:2E:CC:02:20:16:9D:5E:15:79:A0:EB:95:D0:
                                AA:50:11:40:E8:28:34:90:D8:E8:41:02:B5:A8:B0:FB:
                                49:EE:28:DA:16:31:FF
                Signed Certificate Timestamp:
                    Version   : v1 (0x0)
                    Log ID    : 41:C8:CA:B1:DF:22:46:4A:10:C6:A1:3A:09:42:87:5E:
                                4E:31:8B:1B:03:EB:EB:4B:C7:68:F0:90:62:96:06:F6
                    Timestamp : Oct 19 16:24:25.988 2021 GMT
                    Extensions: none
                    Signature : ecdsa-with-SHA256
                                30:45:02:21:00:FE:13:FA:3E:0B:2F:1C:AC:71:06:D1:
                                52:64:83:B3:B9:24:19:D6:A3:E5:22:A0:0F:13:50:04:
                                6C:E0:C7:6D:8E:02:20:04:E2:BC:02:FE:5C:CE:92:35:
                                71:EA:CE:1B:C6:B0:8A:25:E3:55:79:7F:31:FD:8D:29:
                                E4:94:0F:76:EA:1F:2E
    Signature Algorithm: ecdsa-with-SHA384
         30:66:02:31:00:cd:aa:e8:16:18:0b:5e:de:24:bc:44:76:f3:
         a3:e5:1e:a5:03:19:52:7e:fe:57:2c:0b:fe:e2:af:b4:67:3e:
         6a:82:08:36:ce:01:60:30:8b:5b:a3:4e:50:27:1f:de:02:02:
         31:00:8e:fb:20:28:7e:b5:cf:df:1f:90:99:09:83:b0:77:70:
         6f:e0:94:3f:9d:59:7d:ca:6c:21:69:2e:69:d2:cc:0f:e9:ac:
         53:c4:93:c2:9c:d6:83:96:e5:73:39:72:7d:9c
```


### Possible headers in X509 formated file

- PEM_STRING_X509_OLD     "X509 CERTIFICATE"
- PEM_STRING_X509         "CERTIFICATE"
- PEM_STRING_X509_TRUSTED "TRUSTED CERTIFICATE"
- PEM_STRING_X509_REQ_OLD "NEW CERTIFICATE REQUEST"
- PEM_STRING_X509_REQ     "CERTIFICATE REQUEST"
- PEM_STRING_X509_CRL     "X509 CRL"
- PEM_STRING_EVP_PKEY     "ANY PRIVATE KEY"
- PEM_STRING_PUBLIC       "PUBLIC KEY"
- PEM_STRING_RSA          "RSA PRIVATE KEY"
- PEM_STRING_RSA_PUBLIC   "RSA PUBLIC KEY"
- PEM_STRING_DSA          "DSA PRIVATE KEY"
- PEM_STRING_DSA_PUBLIC   "DSA PUBLIC KEY"
- PEM_STRING_PKCS7        "PKCS7"
- PEM_STRING_PKCS7_SIGNED "PKCS #7 SIGNED DATA"
- PEM_STRING_PKCS8        "ENCRYPTED PRIVATE KEY"
- PEM_STRING_PKCS8INF     "PRIVATE KEY"
- PEM_STRING_DHPARAMS     "DH PARAMETERS"
- PEM_STRING_DHXPARAMS    "X9.42 DH PARAMETERS"
- PEM_STRING_SSL_SESSION  "SSL SESSION PARAMETERS"
- PEM_STRING_DSAPARAMS    "DSA PARAMETERS"
- PEM_STRING_ECDSA_PUBLIC "ECDSA PUBLIC KEY"
- PEM_STRING_ECPARAMETERS "EC PARAMETERS"
- PEM_STRING_ECPRIVATEKEY "EC PRIVATE KEY"
- PEM_STRING_PARAMETERS   "PARAMETERS"
- PEM_STRING_CMS          "CMS"

## Django
Django runserver can't serve public-key certificate nor do TLS handshake. Use Gunicorn for that. WSGI generally holds certificate for Django.


## Gunicorn
Gunicorn server serves public-key certificate to client's browser. Nginx optionally can be a reverse-proxy. To use HTTPS public-key certificate needs to be installed for Gunicorn.

Restart Gunicorn with keyfile and certfile:

```
gunicorn app.wsgi --keyfile private_key.pem --certfile certificate.pem
```


### Self-signed certificate
Provides confidentiality without authentication. Easy and cheap to create eg. with ECC or RSA. Common name can be `localhost`. 

Create self-signed certificate and private key:
```
openssl ecparam -list_curves
openssl ecparam -name secp521r1 -genkey -param_enc explicit -out private-key.pem
openssl req -new -x509 -key private-key.pem -out server.pem -days 730
```

Examine the keys with:
```
openssl ecparam -in private-key.pem -text -noout
openssl x509 -in server.pem -text -noout
```

In case specific server requires chained certificate and private key you can cat them together:
```
cat private-key.pem server.pem > server-private.pem
```

With this your connection will go through HTTPS but TLS handshake will fail because self-signed certificate is not trusted by your browser. You need to configure your OS to trust this certificate.

### HSTS
Strict-Transport-Security response header

Following header instructs browser to only accept HTTPS traffic for the next 3600 seconds and do that also for subdomains:
`Strict-Transport-Security: max-age=3600; includeSubDomains`

Can be configured with Django settings `SECURE_HSTS_SECONDS` and `SECURE_HSTS_INCLUDE_SUBDOMAINS`

HSTS does not redirect initial client HTTP request.

### HTTPS redirects
Can be configured with Django settings `SECURE_SSL_REDIRECT`




## Requests
TLS is enabled by default if schema is HTTPS. To keep encryption but skip server authentication use `verify` option:
```
requests.get('https://www.python.org', verify=False)
```
To use custom CA Bundle add path to file or use environmental variable `REQUESTS_CA_BUNDLE`:
```
requests.get('https://www.python.org', verify='/path/to/ca_bundle_file')
```
- CA Bundle allows client (requests library) to authenticate server against CA listed in it.
- CA Bundle - root and intermediate certificates combined in correct order - intermediate first, root second, NO SERVER CERTIFICATE here
- CA Bundle combined with server certificate makes certificate chain (chain of trust)

It's possible to additionally authenticate client by using `cert` tuple with server certificate and private key:
```
url = 'https://www.python.org'
server_cert = ('/path/to/certificate.pem', '/path/to/private_key.pem')
requests.get(url, cert=server_cert)
```

Two-way certificate verification:
```
requests.get(url, server_cert, verify=ca_bundle_path)
```

### Session
```
session = requests.Session()
session.verify=False
cert = ('/path/to/certificate.pem', '/path/to/private_key.pem')
session.cert = cert
```

## Django database
Usually db vendor specific options are available;
```
DATABASES = {
    ...
    "OPTIONS": {
        "sslmode": "verify-full"}
    }
```

## Django emails
In Django set `EMAIL_USE_SSL` to use _Implicit TLS_ for SMTP (SMTPS) that will negotiate TLS connection to be used with `django.core.mail` module. TLS clients may or may not perform server authentication. Django doesn't for emails.

It's possible to also enable client authentication with `EMAIL_SSL_KEYFILE` and `EMAIL_SSL_CERTFILE`. 

Use `EMAIL_HOST_USER` and `EMAIL_HOST_PASSWORD` only when TLS is enabled.
